# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [1]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

A continuación leemos los datasets en dataframes de Pandas.

La siguiente función sirve para correr el código sobre una instancia ejecutando un método en particular.
- FB: Fuerza bruta
- BT: Backtracking con ambas podas.
- BT-F: Backtracking solamente con poda por factibilidad.
- BT-O: Backtracking solamente con poda por optimalidad.
- DP: Programación dinámica.

In [2]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();

df_alto_contagio = pd.read_csv("instancias/alto_contagio/indice.csv");
df_bajo_contagio = pd.read_csv("instancias/bajo_contagio/indice.csv");
df_mejor_caso_bt = pd.read_csv("instancias/mejor-caso-BT/indice.csv");
df_mejor_caso_bt_op = pd.read_csv("instancias/mejor-caso-BT-O/indice.csv");
df_peor_caso_bt = pd.read_csv("instancias/peor-caso-BT/indice.csv");
df_variacion_n_m = pd.read_csv("instancias/variacion_n_m/indice.csv");

## Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

In [3]:
def correr_experimento(metodo, archivo_instancia):
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["../npm", metodo], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if (exit_code != 0): raise(F"Hubo un error en la experimentacion para el algoritmo: {algoritmo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return tiempo_de_ejecucion;

In [4]:
experimentos = [];

### Experimento 1
Correr Fuerza Bruta en las primeras 50 instancias de bajo-contagio y alto_contagio.

In [156]:
for n in range(0, 50):
    fila_n = df_alto_contagio.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["limite_contagio"], "FB", fila_n["archivo"]]);

for n in range(0, 50):
    fila_n = df_bajo_contagio.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["limite_contagio"], "FB", fila_n["archivo"]]);


## Experimento 2
Correr BT para mejor y peor caso.

In [148]:
for n in range(0, df_mejor_caso_bt.shape[0]):
    fila_n = df_mejor_caso_bt.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["limite_contagio"], "BT", fila_n["archivo"]]);

for n in range(0, df_peor_caso_bt.shape[0]):
    fila_n = df_peor_caso_bt.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["limite_contagio"], "BT", fila_n["archivo"]]);


## Experimento 3
Correr BT, BT-F, BT-O para mejor caso, mejor caso con optimalidad y peor caso.

In [199]:
for n in range(0, 50):
    fila_n = df_mejor_caso_bt.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["limite_contagio"], "BT", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], n, fila_n["limite_contagio"], "BT-F", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], n, fila_n["limite_contagio"], "BT-O", fila_n["archivo"]]);

for n in range(0, 50):
    fila_n = df_mejor_caso_bt_op.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["limite_contagio"], "BT", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], n, fila_n["limite_contagio"], "BT-F", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], n, fila_n["limite_contagio"], "BT-O", fila_n["archivo"]]);

for n in range(0, 40):
    fila_n = df_peor_caso_bt.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["limite_contagio"], "BT", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], n, fila_n["limite_contagio"], "BT-F", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], n, fila_n["limite_contagio"], "BT-O", fila_n["archivo"]]);
    

# Experimento 4
Correr DP para el dataset que varía la cantidad de locales y el número de contagios.

In [121]:
for i in range(0, df_variacion_n_m.shape[0]):
    fila = df_variacion_n_m.iloc[i];
    experimentos.append([fila["dataset"], fila["cantidad_locales"], fila["limite_contagio"], "DP", fila["archivo"]]);


## Experimento 5
Correr DP y BT para el dataset _bajo-contagio_ y _alto-contagio_.

In [5]:
for i in range(0, df_bajo_contagio.shape[0]):
    fila = df_bajo_contagio.iloc[i];
    experimentos.append([fila["dataset"], fila["cantidad_locales"], fila["limite_contagio"], "DP", fila["archivo"]]);
    
for i in range(0, df_bajo_contagio.shape[0]):
    fila = df_bajo_contagio.iloc[i];
    experimentos.append([fila["dataset"], fila["cantidad_locales"], fila["limite_contagio"], "BT", fila["archivo"]]);

for i in range(0, df_alto_contagio.shape[0]):
    fila = df_alto_contagio.iloc[i];
    experimentos.append([fila["dataset"], fila["cantidad_locales"], fila["limite_contagio"], "DP", fila["archivo"]]);

for i in range(0, df_alto_contagio.shape[0]):
    fila = df_alto_contagio.iloc[i];
    experimentos.append([fila["dataset"], fila["cantidad_locales"], fila["limite_contagio"], "BT", fila["archivo"]]);


## Ejecutar los experimentos y guardar los resultados en un archivo CSV.
Este paso puede tardar unos minutos hasta terminar de ejecutarse.

In [6]:
columnas = ["dataset", "cantidad_locales", "limite_contagio", "metodo", "tiempo"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    for i in range(0, T):
        tiempos.append(correr_experimento(experimento[3], experimento[4]));
    tiempo = np.median(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado.csv", index=False, header=True);

'Experimento: 896/896'